In [1]:
import minime
import pandas as pd
# python imports
import re
import json
from os.path import join
import cPickle
import statistics
import math

# third party imports
import pandas as pd
import escher
import cobra.test
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import ribosome, tRNA_charging, transcription, translocation, chaperones

from minime.util import dogma
from minime import *
from minime.util import building
from minime.util.mass import compute_RNA_mass
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions
import matplotlib
from tqdm import tqdm
import sys
import os

import copy
from copy import deepcopy
from tqdm import tqdm

import os
import copy


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from TRNA_Workflow_functions_for_enzyme_sweeps import *
#imports all the functions to create tRNA_charging.py and post_transcriptional_modidification.txt output files 

In [3]:
#positive sweeps
def import_modification_network(modification_network_path , filename):
    
    with open(modification_network_path + filename, 'rb') as infile:
        modification_network = json.load(infile)

        #identify the tRNA_enzyme!
        if 'control' in filename:
            tRNA_enzyme = 'control'
        else:
            tRNA_enzyme = filename.split('_positive_ONLY_tRNA_MODOMICS_network_map.json')[0]

                
    return modification_network, tRNA_enzyme

In [4]:
def parse_modification_network_for_tRNA_mods(modification_network):
    import copy
    from copy import deepcopy
    modification_network_copy = copy.deepcopy(modification_network)
    
    for base in modification_network.keys():
        for modification in modification_network[base].keys():
            for position, enzyme in modification_network[base][modification]['enzymes'].items():
                if position == u'20A':
                    continue
                elif int(position) > 76:
                    del  modification_network_copy[base][modification]['enzymes'][position]
                    
            if modification_network_copy[base][modification]['enzymes'] == {}:
                del  modification_network_copy[base][modification]
    
    return modification_network_copy

In [5]:
#positive sweep
def run_workflow_functions_and_output_files(modification_network, rna_type, tRNA_enzyme):
    import copy
    from copy import deepcopy
    #three results per modification_network
    equal_paths_definitions = copy.deepcopy(check_equivalent_RNA_MODOMICS_paths(modification_network, rna_type,'enzymes'))
    sub_path_details = copy.deepcopy(add_carriers_to_rna_paths(modification_network, rna_type))
    print equal_paths_definitions,'eq'
    print sub_path_details, 'sub'
    bnum_modification_dict = copy.deepcopy(add_equivalent_paths_to_bnum_modification_dict(modification_network, rna_type , 'metabolites'))
    
    #create the two relevant outputs
    [trna_charging, df_trna_substrates] = build_trna_charging_AND_post_transcriptional_modification_of_tRNA(bnum_modification_dict , equal_paths_definitions, sub_path_details )
    
    #saving the files
    #save_trna_charging_dictionary_or_post_transciptional_files_positive_sweeps(trna_charging, tRNA_enzyme)
    #save_trna_charging_dictionary_or_post_transciptional_files_positive_sweeps(df_trna_substrates, tRNA_enzyme)
    
    return
        

In [6]:
#positive sweep
def run_workflow_functions_and_output_files(modification_network, rna_type, tRNA_enzyme):
    import copy
    from copy import deepcopy
    #three results per modification_network
    equal_paths_definitions = copy.deepcopy(check_equivalent_RNA_MODOMICS_paths(modification_network, rna_type,'enzymes'))
    sub_path_details = copy.deepcopy(add_carriers_to_rna_paths(modification_network, rna_type))
    #print equal_paths_definitions,'eq'
    #print sub_path_details, 'sub'
    bnum_modification_dict = copy.deepcopy(add_equivalent_paths_to_bnum_modification_dict(modification_network, rna_type , 'metabolites'))
    #print bnum_modification_dict
    #print equal_paths_definitions
    #print sub_path_details
    
    #create the two relevant outputs
    [trna_charging, df_trna_substrates] = build_trna_charging_AND_post_transcriptional_modification_of_tRNA(bnum_modification_dict , equal_paths_definitions, sub_path_details )
    
    #saving the files
    save_trna_charging_dictionary_or_post_transciptional_files_positve_sweeps(trna_charging, tRNA_enzyme)
    save_trna_charging_dictionary_or_post_transciptional_files_positve_sweeps(df_trna_substrates, tRNA_enzyme)
    
    return trna_charging, df_trna_substrates
        

In [7]:
#input the network maps for each enzyme KO!!
modification_network_path = './modified_tRNA_modomics_network_map_for_sweeps/positive_enzyme_sweeps/'
rna_type = 'tRNA'
count = 0 
for filename in tqdm(os.listdir(modification_network_path)):
    if '.json' in filename:
        [modification_network, tRNA_enzyme] = import_modification_network(modification_network_path , filename)
        modification_network = parse_modification_network_for_tRNA_mods(modification_network)
        count = count  + 1
        print '\n', '----------------'
        print '----------------'
        print '----------------'
        
        print count , tRNA_enzyme
        print filename
        print '----------------'
        print '----------------'
        
        run_workflow_functions_and_output_files(modification_network, rna_type, tRNA_enzyme)
        
    
                
        
        
        

  0%|          | 0/38 [00:00<?, ?it/s]


----------------
----------------
----------------
1 TcdA_dim
TcdA_dim_positive_ONLY_tRNA_MODOMICS_network_map.json
----------------
----------------
3A.  create_RNA_dataframe_from_substrates

  3%|▎         | 1/38 [00:04<02:44,  4.45s/it]


saved tRNA_charging_ONLY_TcdA_dim.json 
saved post_transcriptional_modification_of_tRNA_ONLY_TcdA_dim.txt 

----------------
----------------
----------------
2 TmcA_mono
TmcA_mono_positive_ONLY_tRNA_MODOMICS_network_map.json
----------------
----------------
3A.  create_RNA_dataframe_from_substrates
sequence checking DF for Adenine modifications
sequence checking DF for Cytidine modifications
sequence checking DF for Guanine modifications
sequence checking DF for Uridine modifications
		DF length orginal : 4
		DF length after sequence allignment : 4
3C. create_final_bnum_modification_dict

  5%|▌         | 2/38 [00:09<02:48,  4.67s/it]

...


saved tRNA_charging_ONLY_TilS_mono.json 
saved post_transcriptional_modification_of_tRNA_ONLY_TilS_mono.txt 

----------------
----------------
----------------
37 YdaO_mono
YdaO_mono_positive_ONLY_tRNA_MODOMICS_network_map.json
----------------
----------------
3A.  create_RNA_dataframe_from_substrates
sequence checking DF for Adenine modifications
sequence checking DF for Cytidine modifications
sequence checking DF for Guanine modifications
sequence checking DF for Uridine modifications
		DF length orginal : 7
		DF length after sequence allignment : 7
3C. create_final_bnum_modification_dict

 97%|█████████▋| 37/38 [03:55<00:06,  6.73s/it]


saved tRNA_charging_ONLY_YdaO_mono.json 
saved post_transcriptional_modification_of_tRNA_ONLY_YdaO_mono.txt 

----------------
----------------
----------------
38 MnmC_mono_mod_fad
MnmC_mono_mod_fad_positive_ONLY_tRNA_MODOMICS_network_map.json
----------------
----------------
3A.  create_RNA_dataframe_from_substrates

100%|██████████| 38/38 [04:00<00:00,  6.10s/it]


saved tRNA_charging_ONLY_MnmC_mono_mod_fad.json 
saved post_transcriptional_modification_of_tRNA_ONLY_MnmC_mono_mod_fad.txt 


# # troubleshooting